In [1]:
!pip install selenium

In [6]:
#import all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By

# Quetions 1 & 2

In [7]:
driver = webdriver.Chrome()

In [8]:
driver.get('https://www.amazon.in/')

In [9]:
# Find the search bar using its XPath
search_bar = driver.find_element(By.XPATH,'//*[@id="twotabsearchtextbox"]')
# Input keyword to search
search_bar.send_keys('guitar')
# Find the search button and click it
search_button = driver.find_element(By.XPATH,'//*[@id="nav-search-submit-button"]')
search_button.click()


In [10]:
productName=[]

In [11]:
#scraping the Product_Name 
PName=driver.find_elements(By.XPATH,'//span[@class="a-size-base-plus a-color-base a-text-normal"]')
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
print(len(productName),productName)

65 ['Kadence rosewood Guitar Frontier Series, Electric Acoustic Black Guitar With EQ, Die Cast Keys, Set Of Strings, Strap, Picks And Bag (Black EQ, Electric Acoustic)', 'Kadence Frontier guitar with Online Guitar learning course , Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)', 'Kadence Slowhand Premium Jumbo Semi Acoustic Guitar with Heavy Padded Bag, guitar cable, Pro Capo (Black Spruce Wood)', 'Henrix 38C 38 Inch Cutaway Acoustic Guitar With Dual Action Truss Rod, Gigbag, Picks, String Set, String Winder, Strap, Polishing Cloth & Ebook - Black', 'Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black (Acoustic Guitar Kit, Red Sunburst)', 'Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 Inch Cutaway, Right Handed, White with Bag, Strings, Picks and Strap', 'Juârez Lindenwood Acoustic Guitar, 38 Inches Cutaway, JRZ38C With Bag, Strings, Pick And Strap, Blue', 'Juârez Acoustic Guitar, 38 Inch Cutawa

In [12]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element(By.XPATH,'//span[@class="s-pagination-item s-pagination-disabled"]').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [13]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['URL']=[]

In [21]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element(By.XPATH,'//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element(By.XPATH,'//span[@class="a-size-base po-break-word"]')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
   
    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/span')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
   
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4MjY0MjIyMTk0MjIyMTExOjE3MTM2NzYzNjI6c3BfYXRmOjMwMDAxNTUzNzk3OTIzMjo6MDo6&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB078GTJP5Y%2Fref%3Dsr_1_1_sspa%3Fcrid%3DQ8DG54OMTRQP%26dib%3DeyJ2IjoiMSJ9.sKiWAQwvKMcgTtHqXgOVFD7jHk_6yw8lhmM4dCJGqglN5lcYwRPN5Cbj9MQ59VfU-X4AL4ngFv0QZpR8xdbDKwcICNTTTApzgW7lkYN087izCcIgZj8zjZdxy04VllVvGPdHpVLpWdZ3Yg8Gkuuv6nfE8DVt8KFuff7NTQO9boVbWBzkCjwEqgaSBbalwWFhntS_tDeoPcs73ceqKd8SdR1n1XONaohuHYffZHxVKjwSTzfjYai4ZwKQsMjFGU0GwO7bRO6J0VTz_8OxuzoWPaFPuyV61VEOu2vrypA9cqQ.8AZyDiPJnOt2n0wlLq1oizjsqDRPihTf6xG6VJm5_EI%26dib_tag%3Dse%26keywords%3Dguitar%26qid%3D1713676362%26sprefix%3D%252Caps%252C222%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4MjY0MjIyMTk0MjIyMTExOjE3MTM2NzYzNjI6c3BfYXRmOjIwMDk2NjA4NzUwNDk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_2_sspa%3Fcrid%3DQ8DG54OMTRQP

In [22]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df


,Brand,Name,Price,Return/Exchange,Expected Delivery,Availability,URL
0,Visit the Kadence Store,-,"4,899",Free Delivery,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Visit the Kadence Store,-,"4,799",Free Delivery,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the Kadence Store,-,"9,999",Free Delivery,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Visit the Henrix Store,-,"3,199",Free Delivery,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Visit the Kadence Store,"Kadence rosewood Guitar Frontier Series, Elect...","4,899",Free Delivery,"Tuesday, 23 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
5,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,"4,799",Free Delivery,"Tuesday, 23 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
6,Visit the Kadence Store,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,999",Free Delivery,"Tuesday, 23 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
7,Visit the Henrix Store,Henrix 38C 38 Inch Cutaway Acoustic Guitar Wit...,"3,199",Free Delivery,"Thursday, 25 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
8,Visit the Kadence Store,Kadence,"4,899",Free Delivery,"Tuesday, 23 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
9,Visit the Kadence Store,Kadence,"4,799",Free Delivery,"Tuesday, 23 April",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


# Question.3


In [24]:
driver = webdriver.Chrome()

In [25]:
driver.get('https://images.google.com/')

In [27]:
search_bar = driver.find_element(By.XPATH,'//*[@id="APjFqb"]')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [28]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 1000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,1000)")

start scrolling to generate more images on the page...


In [36]:
 images = driver.find_elements(By.XPATH,'//img[@class="YQ4gaf"]')

In [37]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

378

In [39]:
import requests


In [47]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("C:\\Users\\Sumit Kaushik\\OneDrive\\img"+str(i)+".jpg", "wb")
    file.write(response.content)
    



# Question.4

In [48]:
driver = webdriver.Chrome()

In [49]:
url4="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url4)

In [50]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [52]:
#scraping the Brand_Name 
BName=driver.find_elements(By.XPATH,'//div[@class="KzDlHZ"]')
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['vivo T3x 5G (Celestial Green, 128 GB)', 'Motorola G34 5G (Ice Blue, 128 GB)', 'Motorola G34 5G (Charcoal Black, 128 GB)', 'Motorola G34 5G (Ocean Green, 128 GB)', 'MOTOROLA g24 Power (Ink blue, 128 GB)', 'REDMI 13C (Stardust Black, 128 GB)', 'POCO M6 Pro 5G (Power Black, 128 GB)', 'MOTOROLA g24 Power (Ink blue, 128 GB)', 'SAMSUNG Galaxy F14 5G (B.A.E. Purple, 128 GB)', 'POCO M6 Pro 5G (Forest Green, 128 GB)', 'MOTOROLA G04 (Sea Green, 128 GB)', 'POCO M6 5G - Locked with Airtel Prepaid (Polaris Green, 128 GB)', 'MOTOROLA g24 Power (Glacier Blue, 128 GB)', 'REDMI 13C (Starshine Green, 128 GB)', 'MOTOROLA G04 (Sea Green, 64 GB)', 'POCO M6 Pro 5G (Forest Green, 128 GB)', 'POCO M6 Pro 5G (Power Black, 128 GB)', 'POCO C65 (Pastel Blue, 128 GB)', 'Motorola g54 5G (Pearl Blue, 128 GB)', 'Motorola Edge 40 Neo (Soothing Sea, 256 GB)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256 GB)', 'Motorola G34 5G (Ocean Green, 128 GB)', 'MOTOROLA G04 (Satin Blue, 64 GB)', 'OnePlus Nord CE 3 Lite 5G 

In [53]:
#scraping the Storage_RAM_ROM 
ram=driver.find_elements(By.XPATH,'//ul[@class="G4BRas"]//li[1]')
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '12 GB RAM | 256 GB ROM', '8 GB RAM | 256 GB ROM', '4 GB RAM | 128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM']


In [54]:
#scraping the P_F_Camera 
PC=driver.find_elements(By.XPATH,'//ul[@class="G4BRas"]//li[3]')
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

24 ['50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP Rear Camera | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP + 2MP | 13MP Front Camera', '50MP + 2MP | 8MP Front Camera', '16MP Rear Camera | 5MP Front Camera', '50MP Rear Camera | 5MP Front Camera', '50MP + 2MP | 16MP Front Camera', '50MP Rear Camera | 8MP Front Camera', '16MP Rear Camera | 5MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP + AI Lens + 2MP | 8MP Front Camera', '50MP (OIS) + 8MP | 16MP Front Camera', '50MP + 13MP | 32MP Front Camera', '108MP Rear Camera', '50MP + 2MP | 16MP Front Camera', '16MP Rear Camera | 5MP Front Camera', '108MP Rear Camera']


In [55]:
#scraping the Display_size_Resolution 
DS=driver.find_elements(By.XPATH,'//ul[@class="G4BRas"]//li[2]')
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

24 ['17.07 cm (6.72 inch) Full HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '17.12 cm (6.74 inch) HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '17.12 cm (6.74 inch) HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '17.12 cm (6.74 inch) HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '17.25 cm (6.79 inch) Full HD+ Display', '17.12 cm (6.74 inch) HD+ Display', '16.51 cm (6.5 inch) Full HD+ Display', '16.64 cm (6.55 inch) Full HD+ Display', '17.07 cm (6.72 inch) Display', '16.51 cm (6.5 inch) HD+ Display', '16.76 cm (6.6 inch) HD+ Display', '17.07 cm (6.72 inch) Display']


In [56]:
#scraping the ProcessorAndCores 
P=driver.find_elements(By.XPATH,'//ul[@class="G4BRas"]//li[5]')
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

24 ['6 Gen 1 Processor', 'Snapdragon 695 5G Processor', 'Snapdragon 695 5G Processor', 'Snapdragon 695 5G Processor', 'Helio G85 Processor', 'Helio G85 Processor', 'Snapdragon 4 Gen 2 Processor', 'Helio G85 Processor', 'Exynos 1330, Octa Core Processor', 'Snapdragon 4 Gen 2 Processor', 'Unisoc T606 Processor', 'Mediatek Dimensity 6100+ Processor', 'Helio G85 Processor', 'Helio G85 Processor', 'Unisoc T606 Processor', 'Snapdragon 4 Gen 2 Processor', 'Snapdragon 4 Gen 2 Processor', 'Helio G85 Processor', 'Dimensity 7020 Processor', 'Dimensity 7030 Processor', 'Domestic Warranty of 12 months on phone & 6 months on accessories', 'Snapdragon 695 5G Processor', 'Unisoc T606 Processor', 'Domestic Warranty of 12 months on phone & 6 months on accessories']


In [57]:
#scraping the Battery 
B=driver.find_elements(By.XPATH,'//ul[@class="G4BRas"]//li[4]')
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

24 ['6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '6000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery', '5000 mAh Battery']


In [59]:
#scraping the Price 
price=driver.find_elements(By.XPATH,'//div[@class="Nx9bqj _4b5DiR"]')
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

24 ['₹13,499', '₹11,999', '₹11,999', '₹11,999', '₹8,999', '₹7,699', '₹9,999', '₹7,999', '₹10,490', '₹9,999', '₹7,999', '₹8,599', '₹8,999', '₹7,699', '₹6,999', '₹10,999', '₹10,999', '₹7,999', '₹14,999', '₹24,999', '₹19,406', '₹10,999', '₹6,999', '₹17,399']


In [60]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Amount P_F_Camera']=P_F_Camera
FlipKart['Display_size_Resolution']=Display_size_Resolution
FlipKart['ProcessorAndCores']=ProcessorAndCores
FlipKart['Battery']=Battery
FlipKart['Price']=Price

FlipKart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"vivo T3x 5G (Celestial Green, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 8MP Front Camera,17.07 cm (6.72 inch) Full HD+ Display,6 Gen 1 Processor,6000 mAh Battery,"₹13,499"
1,"Motorola G34 5G (Ice Blue, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 16MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Snapdragon 695 5G Processor,5000 mAh Battery,"₹11,999"
2,"Motorola G34 5G (Charcoal Black, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 16MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Snapdragon 695 5G Processor,5000 mAh Battery,"₹11,999"
3,"Motorola G34 5G (Ocean Green, 128 GB)",8 GB RAM | 128 GB ROM,50MP + 2MP | 16MP Front Camera,16.51 cm (6.5 inch) HD+ Display,Snapdragon 695 5G Processor,5000 mAh Battery,"₹11,999"
4,"MOTOROLA g24 Power (Ink blue, 128 GB)",8 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 16MP Front Camera,16.76 cm (6.6 inch) HD+ Display,Helio G85 Processor,6000 mAh Battery,"₹8,999"
5,"REDMI 13C (Stardust Black, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP Rear Camera | 8MP Front Camera,17.12 cm (6.74 inch) HD+ Display,Helio G85 Processor,5000 mAh Battery,"₹7,699"
6,"POCO M6 Pro 5G (Power Black, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 8MP Front Camera,17.25 cm (6.79 inch) Full HD+ Display,Snapdragon 4 Gen 2 Processor,5000 mAh Battery,"₹9,999"
7,"MOTOROLA g24 Power (Ink blue, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 16MP Front Camera,16.76 cm (6.6 inch) HD+ Display,Helio G85 Processor,6000 mAh Battery,"₹7,999"
8,"SAMSUNG Galaxy F14 5G (B.A.E. Purple, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 13MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,"Exynos 1330, Octa Core Processor",6000 mAh Battery,"₹10,490"
9,"POCO M6 Pro 5G (Forest Green, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 8MP Front Camera,17.25 cm (6.79 inch) Full HD+ Display,Snapdragon 4 Gen 2 Processor,5000 mAh Battery,"₹9,999"


# Question.5

In [8]:
#import all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By

In [9]:
driver = webdriver.Chrome()

In [10]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element(By.XPATH,'//input[@class="searchboxinput xiQnY"]')                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element(By.XPATH,'//span[@class="google-symbols"]')               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : delhi
URL Extracted:  https://www.google.co.in/maps/place/Delhi/@27.1381927,80.8593041,7z/data=!4m6!3m5!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902!16zL20vMDlmMDc?entry=ttu
Latitude = 27.1381927, Longitude = 80.8593041


# Question.6

In [11]:
driver = webdriver.Chrome()

In [12]:
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [13]:
driver.get(url)

In [14]:
Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [18]:
br=driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]')
len(br)

10

In [19]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

['MSI Titan GT77 HX 13VI-092IN 13th Gen Core i9-13980HX',
 'ASUS ROG Strix SCAR 16 G634JYR-RA001WS',
 'MSI Raider GE78HX 13VI-087IN 13th Gen Core i9-13980HX',
 'ASUS ROG Strix SCAR 18 G834JY-N6056WS',
 'Lenovo Legion Pro 7i',
 'OMEN by HP 17-ck2004TX',
 'Dell Alienware M18 R1',
 'Dell Alienware X16 R1 (2023)',
 'MSI Raider GE67HX 12UHS-057IN 12th Gen Core i9-12900HX',
 'Acer Predator Helios 16']

In [27]:
sp=driver.find_elements(By.XPATH,'//div[@class="cegg-no-top-margin cegg-list-logo-title"]')
len(sp)

9

In [28]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

['MSI Titan GT77 HX, Intel 13th Gen. i9-13980HX, 44CM FHD 144Hz Mini LED, HDR 1000 Gaming Laptop...',
 'ASUS ROG Strix Scar 16, Intel® Core™ i9 14900HX 14th Gen, 16"(40.64cm) 2.5K QHD+240Hz, Gaming...',
 'MSI Raider GE78HX, Intel 13th Gen. i9-13980HX, 43CM QHD+ 240Hz Gaming Laptop (64GB/2TB NVMe...',
 'ASUS ROG Strix Scar 18 (2023), Intel Core i9 13th Gen, 18" (45.72 cm) QHD+ 240Hz/3ms, 16GB RTX...',
 'Lenovo Legion Pro 7i 16" QHD+ 500nits Gaming Laptop 240Hz Intel Core i9-13900HX 16GB RAM 1TB SSD...',
 'HP OMEN Gaming Laptop, 13th Gen Intel Core i9-13900HX, 12GB RTX 4080 GPU, 17.3-inch (43.9 cm),...',
 'Dell Alienware X16 R1 Gaming, Intel Core i9-13900HK/32GB/1TB SSD/NVIDIA RTX 4080 12GB GDDR6/16.0"...',
 'MSI Raider GE67HX, Intel 12th Gen. i9-12900HX, 40CM QHD OLED 240Hz Gaming Laptop (32GB/2TB NVMe...',
 'Acer Predator Helios 16 Gaming Laptop (Intel Core i9 13900HX/32 GB Ram/1 TB SSD/RTX 4080...',
 'Key SpecificationsOperating System (WithWindows 11 HomeDisplay Size (In Inches)17.3P

In [38]:
des=driver.find_elements(By.XPATH,'//div[@class="egg-container cegg-list-withlogos"]')
len(des)

9

In [39]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))
Products_Description

['Not very far behind the Titan you have the ASUS ROG Strix SCAR 16. This gaming laptop is powered by the newer 14th gen Intel Core i9-14900HX, which has the same number of cores as the 13980HX, but slightly faster clock speeds. This laptop also features a NVIDIA GeForce RTX 4090 Laptop GPU with 16GB of VRAM. It has a 16-inch QHD+ (2560 x 1600, WQXGA) ROG Nebula HDR Display with a Mini LED Panel and a 240Hz refresh rate. This is one of the best gaming laptop displays we’ve tested to date, with 100 per cent sRGB and DCI-P3 coverage. The laptop comes with 32GB DDR5 RAM, which is expandable up to 64GB. It has a 2TB PCIe 4.0 NVMe M.2 Performance SSD for storage. The laptop includes a variety of I/O ports, including HDMI 2.1, USB 3.2 Gen 2 Type-A and Type-C, 2.5G LAN port, and Thunderbolt 4.',
 'Not very far behind the Titan you have the ASUS ROG Strix SCAR 16. This gaming laptop is powered by the newer 14th gen Intel Core i9-14900HX, which has the same number of cores as the 13980HX, but s

In [41]:
pri=driver.find_elements(By.XPATH,'//div[@class="cegg-price cegg-price-color cegg-price-instock"]')
len(pri)

4

In [42]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

['Rs. 329,990', 'Rs. 249,990', 'Rs. 386,490', 'Rs. 247,990']

In [45]:
digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:4]
digit_lap['Price']=Price[0:4]
digit_lap['Specification']=Specification[0:4]
digit_lap['Description']=Products_Description[0:4]
digit_lap

,Brands,Price,Specification,Description
0,MSI Titan GT77 HX 13VI-092IN 13th Gen Core i9-...,"Rs. 329,990","MSI Titan GT77 HX, Intel 13th Gen. i9-13980HX,...",Not very far behind the Titan you have the ASU...
1,ASUS ROG Strix SCAR 16 G634JYR-RA001WS,"Rs. 249,990","ASUS ROG Strix Scar 16, Intel® Core™ i9 14900H...",Not very far behind the Titan you have the ASU...
2,MSI Raider GE78HX 13VI-087IN 13th Gen Core i9-...,"Rs. 386,490","MSI Raider GE78HX, Intel 13th Gen. i9-13980HX,...","MSI Titan GT77 HX, Intel 13th Gen. i9-13980HX,..."
3,ASUS ROG Strix SCAR 18 G834JY-N6056WS,"Rs. 247,990","ASUS ROG Strix Scar 18 (2023), Intel Core i9 1...","ASUS ROG Strix Scar 16, Intel® Core™ i9 14900H..."


# Question.7

In [46]:
driver = webdriver.Chrome()

In [51]:
url="https://www.forbes.com/the details for all billionaires"

In [52]:
driver.get(url)

In [81]:
ranks=[]
names=[]


In [82]:
# scraping rank from the given page
rank_tags=driver.find_elements(By.XPATH,'//div[@class="Table_rank__X4MKf"]/div')
for i in rank_tags:
    rank=i.text
    ranks.append(rank)
rank[:10]

# scraping name from the given page 
name_tags=driver.find_elements(By.XPATH,'//div[@class="Table_personName__Bus2E"]')
for i in name_tags:
    name=i.text
    names.append(name)    
name[:10]

 

[]

In [79]:
print(len(rank[:10]),len(name[:10]))

0 0


In [80]:
import pandas as pd
df=pd.DataFrame({'Rank':rank[:10],'Name':name[:10]})
df

,Rank,Name


# Question.8

In [93]:
from selenium import webdriver
import time
# Let's now import all the required libraries.
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By

In [94]:
# Set up the WebDriver
driver = webdriver.Chrome()

In [95]:
# Open the YouTube video
video_url = 'https://www.youtube.com/watch?v=your_video_id'  # Replace with the URL of the YouTube video
driver.get(video_url)

In [96]:
# Scroll to load comments
scroll_pause_time = 2  # Adjust the pause time as needed
scrolls = 10  # Adjust the number of scrolls as needed


In [97]:
for _ in range(scrolls):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(scroll_pause_time)


In [98]:
# Extract comments, upvotes, and time
comments = driver.find_elements(By.XPATH,'//span[@class="style-scope yt-formatted-string"]')
upvotes = driver.find_elements(By.XPATH,'//div[@class="yt-spec-touch-feedback-shape__fill"]')
times = driver.find_elements(By.XPATH,'//span[@class="bold style-scope yt-formatted-string"]')

In [99]:
# Store the extracted data
extracted_data = []
for comment, upvote, time in zip(comments, upvotes, times):
    extracted_data.append({
    'comment': comment.text,
    'upvote': upvote.text,
    'time': time.text
  }) 

In [100]:
# Close the WebDriver
driver.quit()


In [101]:
# Print the extracted data
for data in extracted_data:
    print(data)

{'comment': '', 'upvote': '', 'time': '27M views'}
{'comment': '', 'upvote': '', 'time': '  '}
{'comment': '', 'upvote': '', 'time': '4 months ago'}
{'comment': '', 'upvote': '', 'time': '  '}
{'comment': 'balaji pasheme', 'upvote': '', 'time': ''}
{'comment': ' · ', 'upvote': '', 'time': ''}
{'comment': 'Playlist', 'upvote': '', 'time': ''}
{'comment': 'Vanshika Rajvanshi', 'upvote': '', 'time': ''}
{'comment': ' · ', 'upvote': '', 'time': ''}
{'comment': 'Playlist', 'upvote': '', 'time': ''}
{'comment': '19,238', 'upvote': '', 'time': ''}
{'comment': 'Comments', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote': '', 'time': ''}
{'comment': '', 'upvote':

# Question.9

In [102]:
driver = webdriver.Chrome()

In [103]:
url='https://www.hostelworld.com/'

In [104]:
driver.get(url)

In [118]:
hostel_names=[]
distancecity_centre=[]
ratings=[]
total_reviews=[]
privates_price=[]
dorms_price=[]
facilities=[]
property_description=[]


In [120]:
 for i in range(3):
    hostel_names=driver.find_elements(By.XPATH,'//div[@class="property-name"]')
    distancecity_centre=driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
    ratings=driver.find_elements(By.XPATH,'//span[@class="number"]')
    
    for j  in hostel_name:
        hostel_name.append(j.text)
    
    for k in distancecity_centr:
        distancecity_centr.append(k.text)
    for l in ratings:
        ratings.append(l.text)
     

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=123.0.6312.124)
Stacktrace:
	GetHandleVerifier [0x00007FF761787032+63090]
	(No symbol) [0x00007FF7616F2C82]
	(No symbol) [0x00007FF76158EC65]
	(No symbol) [0x00007FF76157C31B]
	(No symbol) [0x00007FF76157BEC0]
	(No symbol) [0x00007FF761590EC1]
	(No symbol) [0x00007FF761612AE9]
	(No symbol) [0x00007FF7615F6D83]
	(No symbol) [0x00007FF7615C83A8]
	(No symbol) [0x00007FF7615C9441]
	GetHandleVerifier [0x00007FF761B825AD+4238317]
	GetHandleVerifier [0x00007FF761BBF70D+4488525]
	GetHandleVerifier [0x00007FF761BB79EF+4456495]
	GetHandleVerifier [0x00007FF761860576+953270]
	(No symbol) [0x00007FF7616FE54F]
	(No symbol) [0x00007FF7616F9224]
	(No symbol) [0x00007FF7616F935B]
	(No symbol) [0x00007FF7616E9B94]
	BaseThreadInitThunk [0x00007FFA7999257D+29]
	RtlUserThreadStart [0x00007FFA7B48AA48+40]
